In [1]:
import numpy as np

In [2]:
import os

clip_folder1 = r'./mean_var_kurt_skew'
data = []
labels=  []

for filename in os.listdir(clip_folder1):
    fname = clip_folder1 + '/' + filename
    labels.append(filename[-5])
    x = np.load(fname)
    data.append(x)

In [3]:
data = np.array(data)
labels = np.array(labels)
print(data.shape)
print(labels.shape)

(5587, 512)
(5587,)


### Classifiers

gradient boosting

In [15]:
#loss= deviance

from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse')


In [16]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, data, labels, cv=5)
print(scores)

[0.83542039 0.86762075 0.84780662 0.84959714 0.85944494]


In [17]:
 print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.85 (+/- 0.02)


Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, data, labels, cv=5)
print(scores)

[0.81842576 0.85420394 0.84601611 0.84870188 0.83348254]


In [10]:
 print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.84 (+/- 0.03)


SVM

In [11]:
from sklearn import svm

clf = svm.SVC(kernel = 'poly',verbose = 1,coef0=25)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, data, labels, cv=5)
print(scores)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][0.8059034  0.81484794 0.81468218 0.81557744 0.82273948]


In [12]:
 print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.81 (+/- 0.01)


In [18]:
import numpy as np
import os
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import OneHotEncoder

In [19]:
clip_folder1 = r'./mean_var_kurt_skew'
data = []
labels = []

In [20]:
# Load data and labels
for filename in os.listdir(clip_folder1):
    fname = os.path.join(clip_folder1, filename)
    labels.append(filename[-5])
    x = np.load(fname)
    data.append(x)

In [21]:
# Convert labels to integers (assuming labels are single characters like '0', '1', '2')
labels = np.array(labels).astype(int)

In [22]:
# Initialize classifiers
gb_clf = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse')
rf_clf = RandomForestClassifier()
svm_clf = svm.SVC(kernel='poly', verbose=1, coef0=25)

In [26]:
def get_last_layer_representations(clf, data, labels, cv=5):
    # Use cross_val_predict to get predictions (last layer representations)
    if hasattr(clf, 'predict_proba'):
        predictions = cross_val_predict(clf, data, labels, cv=cv, method='predict_proba')
    else:
        # For SVM, since predict_proba is not available, we use decision function and scale it
        decision_values = cross_val_predict(clf, data, labels, cv=cv, method='decision_function')
        # Scale decision values to [0, 1] range
        predictions = (decision_values - decision_values.min()) / (decision_values.max() - decision_values.min())
    return predictions

In [ ]:
# Get last layer representations from each classifier
gb_last_layer = get_last_layer_representations(gb_clf, data, labels)
rf_last_layer = get_last_layer_representations(rf_clf, data, labels)

In [27]:
svm_last_layer = get_last_layer_representations(svm_clf, data, labels)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [28]:
# Concatenate the last layer representations into one feature vector
combined_features = np.concatenate((gb_last_layer, rf_last_layer, svm_last_layer), axis=1)

In [29]:
# Print shape of combined feature vector
print("Shape of combined feature vector:", combined_features.shape)

Shape of combined feature vector: (5587, 9)


In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [31]:
# Define a simple neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(combined_features.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification, adjust output size accordingly
])


In [32]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Adjust loss function based on your problem type
              metrics=['accuracy'])

In [33]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1280      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9601 (37.50 KB)
Trainable params: 9601 (37.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Train the model using the concatenated feature vector as input
model.fit(combined_features, labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


140/140 [==============================] - 1s 3ms/step - loss: 0.1726 - accuracy: 0.8096 - val_loss: -0.2762 - val_accuracy: 0.8542
Epoch 2/10
140/140 [==============================] - 0s 1ms/step - loss: -1.1657 - accuracy: 0.8156 - val_loss: -2.7669 - val_accuracy: 0.8488
Epoch 3/10
140/140 [==============================] - 0s 1ms/step - loss: -7.3743 - accuracy: 0.8161 - val_loss: -12.7830 - val_accuracy: 0.8148
Epoch 4/10
140/140 [==============================] - 0s 1ms/step - loss: -26.0723 - accuracy: 0.8082 - val_loss: -35.6432 - val_accuracy: 0.7871
Epoch 5/10
140/140 [==============================] - 0s 1ms/step - loss: -64.0299 - accuracy: 0.8073 - val_loss: -81.2374 - val_accuracy: 0.7844
Epoch 6/10
140/140 [==============================] - 0s 1ms/step - loss: -125.5089 - accuracy: 0.8029 - val_loss: -150.8079 - val_accuracy: 0.8050
Epoch 7/10
140/140 [==============================] - 0s 1ms/step - loss: -215.3486 - accuracy: 0.8040 - val_loss: -242.2254 -